# Análise de sentimento de reviews do app do spotify

## Objetivo

Esse projeto tem que como objetivo fazer uma análise sentimental das avaliações do aplicativo do Spotify na Google Play Store

## Dados

Os dados estão presentes em reviews.csv

In [1]:
import pandas as pd
data = pd.read_csv('reviews.csv')
data.head() 

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2,NaN
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1,NaN
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0,NaN
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1,NaN
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1,NaN


Na tabela acima podemos ver as colunas (atributos) dos registros, no caso temos o `Time_Submitted`, `Review`, `Rating`, `Total_thumbsup` e `Reply` que são respectivamente a data e hora que submeteram a avaliação, o texto, a nota que deram que vai de 1 a 5, o tanto que a avaliação recebeu de feedback positivo, e Reply seria as respostas que a review recebeu.

## Metodologia

Cada registro nos dados acima contém dois itens importantes para cumprirmos o objetivo, o texto a ser analisado e a nota dada. Sabendo que faremos uma análise de sentimento da avaliação divididas nas categorias de positiva, neutro e negativa, é esperado que reviews positivas possuem um `Rating` mais alto (4 ou 5) enquanto as negativas mais baixas (1 ou 2).

## Execução

In [2]:
from time import sleep
from textblob import TextBlob
from newspaper import Article
from tqdm.auto import tqdm
import nltk
import pandas as pd

data = pd.read_csv('reviews.csv')
reviews = pd.DataFrame(columns=["index", "Review", "Rating", "sentimento_pontuacao"])

for row_index, registro in tqdm(iterable=data.iterrows(),total=data.shape[0]):
    
    indice = row_index
    review_txt = registro["Review"]     # texto da avaliação
    nota = registro["Rating"]           # de 1 a 5

    blob = TextBlob(review_txt)
    sentimento = blob.sentiment.polarity
    review = pd.DataFrame([[indice, review_txt, nota, sentimento]], columns=["index", "Review", "Rating", "sentimento_pontuacao"])
    reviews = pd.concat([reviews, review])
    

 54%|█████▍    | 33364/61594 [00:54<00:46, 611.22it/s]


KeyboardInterrupt: 